In [1]:
from pathlib import Path

import pandas as pd

In [ ]:
LOG_PATH = Path("..", "log").with_suffix(".jsonl")

In [4]:
def get_all_files_matching(**kwargs):
    """
    Extract all rows where the kwargs are matched.
    """
    log_files = pd.read_json(LOG_PATH, lines=True)
    for key, value in kwargs.items():
        log_files = log_files[log_files[key] == value]
    return log_files

In [ ]:
get_all_files_matching(
    model_name="/public/hf/models/meta-llama/Meta-Llama-3.1-8B-Instruct/",
    split="answerable",
)

,model_name,split,num_samples,use_tools,save,description,debug,template,filename,timestamp
0,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,AverageProperty,eval/runs/2025-05-08T19:25:53.jsonl,2025-05-08 19:25:53
1,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,CountryPropertyComparison,eval/runs/2025-05-08T20:26:23.jsonl,2025-05-08 20:26:23
2,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,CountryThresholdCount,eval/runs/2025-05-08T21:21:38.jsonl,2025-05-08 21:21:38
3,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,FactorIncreaseComparison,eval/runs/2025-05-08T21:53:16.jsonl,2025-05-08 21:53:16
4,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,IncreasePropertyComparison,eval/runs/2025-05-08T22:07:37.jsonl,2025-05-08 22:07:37
5,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,PropertyOfSubject,eval/runs/2025-05-08T23:06:50.jsonl,2025-05-08 23:06:50
6,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,RegionComparison,eval/runs/2025-05-08T23:14:12.jsonl,2025-05-08 23:14:12
7,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,RegionComparisonResult,eval/runs/2025-05-08T23:56:01.jsonl,2025-05-08 23:56:01
8,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,TopNTotal,eval/runs/2025-05-09T00:30:40.jsonl,2025-05-09 00:30:40
9,/public/hf/models/meta-llama/Meta-Llama-3.1-8B...,answerable,100,full,True,,False,TotalProperty,eval/runs/2025-05-09T01:54:16.jsonl,2025-05-09 01:54:16


In [ ]:
row = data.sample(1)

In [ ]:
row["messages"].to_dict()

{8: [{'role': 'system',
   'content': "You are a helpful assistant tasked with answering questions that require multiple intermediate steps of reasoning to arrive at a final answer.\n\nThe questions involve using World Bank data for various countries and indicators, and you have access to a set of tools that can help you retrieve and process this data.\n\nStep-by-step, use the tools provided to execute that plan. Use the `think` tool to think aloud about the steps you will take to solve the problem. Use the `final_answer` tool to return your final answer.\n\nUse the results of each tool call to inform your next step. Note that passing tool calls as the arguments to other tool calls is not allowed. Instead, execute each tool call separately and use the results to perform subsequent calls.\n\nIf a tool call fails, use the error message to help you debug the issue, re-plan, and try again if possible.\n\nBear in mind that some data may not be available for certain countries, indicators, or

In [ ]:
def get_model_final_answer(row):
    # Get the final answer from the row
    messages = row["messages"].to_dict()[row.index[0]]
    # Get the messages where 'role' = 'tool' and 'name' = 'final_answer'
    final_answer = [
        msg
        for msg in messages
        if msg["role"] == "tool" and msg["name"] == "final_answer"
    ]
    if final_answer:
        # Get the content of the final answer
        final_answer_content = final_answer[0]["content"]
        # Convert the content to a string
        final_answer_str = str(final_answer_content)
        return final_answer_str
    else:
        # If no final answer is found, return None
        return None


def get_gold_final_answer(row):
    # Get the final answer from the actions sequence
    actions = row["actions"].to_dict()[row.index[0]]
    # Get the actions where 'name' = 'final_answer'
    final_answer = [action for action in actions if action["name"] == "final_answer"]
    return final_answer["result"]


def get_model_tool_calls(row):
    # Get the tool calls from the row
    messages = row["messages"].to_dict()[row.index[0]]
    # Get the messages where 'role' = 'tool'
    tool_calls = [
        msg["tool_calls"][0]["function"]
        for msg in messages
        if msg["role"] == "assistant" and "tool_calls" in msg
    ]
    return tool_calls


def get_gold_tool_calls(row):
    # Get the tool calls from the actions sequence
    actions = row["actions"].to_dict()[row.index[0]]
    return actions